# 文書分類ハンズオン

## 機械学習モデル開発のワークフローと本章で扱う内容

## 文書分類ハンズオン

### 本章で扱うツールの解説

#### Hugging Face

### データセットの準備

#### livedoor ニュースコーパス

今回は日本語における自然言語処理の試験用データセットとしてしばしば用いられる「livedoor ニュースコーパス」を用います。
livedoorニュースはもともと株式会社ライブドアが運営するニュースサイトでしたが、株式会社ライブドアが旧ハンゲームジャパン株式会社であるNHN Japan株式会社に買収され、現在はNHN Japanが社名変更したLINE株式会社により運営されています。
livedoorニュースの記事の一部には「クリエイティブ・コモンズライセンス『表示 – 改変禁止』」が適用されており、営利目的を含めて再配布可能となっています。
該当するニュース記事を株式会社ロンウイットが収集し、HTMLタグの除去などクリーニングを施した状態で公開しているのが「livedoor　ニュースコーパス」です。
livedoor ニュースコーパスは以下のリンクよりダウンロード可能です。

https://www.rondhuit.com/download.html#ldcc



#### ラベリング

### 前処理

#### 形態素解析
#### ストップワード除去

### 文書分類モデル

### 評価と比較

### モデルのデプロイ

## まとめ

In [1]:
import torch